# Use Case: HLCA Datasets

In [1]:
import sys
sys.path.append("../..")

In [2]:
import os
import time
import scanpy as sc
import idtrack
import pickle

In [3]:
%load_ext autoreload
%autoreload 2

Initialize the graph and pathfinder

In [4]:
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"
idt = idtrack.API(local_repository=local_dir)
idt.configure_logger()
idt.initialize_graph(organism_name='homo_sapiens', ensembl_release=107, return_test=True)

2023-02-15 12:30:00 INFO:graph_maker: The graph is being read.


In [5]:
idt.calculate_graph_caches() 

2023-02-15 12:30:50 INFO:the_graph: Cached properties being calculated: combined_edges
2023-02-15 12:31:23 INFO:the_graph: Cached properties being calculated: combined_edges_assembly_specific_genes
2023-02-15 12:31:26 INFO:the_graph: Cached properties being calculated: combined_edges_genes
2023-02-15 12:31:37 INFO:the_graph: Cached properties being calculated: lower_chars_graph
2023-02-15 12:31:38 INFO:the_graph: Cached properties being calculated: get_active_ranges_of_id
2023-02-15 12:31:57 INFO:the_graph: Cached properties being calculated: available_external_databases
2023-02-15 12:31:58 INFO:the_graph: Cached properties being calculated: external_database_connection_form
2023-02-15 12:32:32 INFO:the_graph: Cached properties being calculated: available_genome_assemblies
2023-02-15 12:32:33 INFO:the_graph: Cached properties being calculated: available_external_databases_assembly
2023-02-15 12:32:34 INFO:the_graph: Cached properties being calculated: node_trios


In [15]:
local_dir_arc = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp_arch"
idt_arc = idtrack.API(local_repository=local_dir_arc)
idt_arc.configure_logger()
idt_arc.initialize_graph(organism_name='homo_sapiens', ensembl_release=107, return_test=True)

2023-02-15 12:37:01 INFO:graph_maker: The graph is being read.


In [16]:
idt_arc.calculate_graph_caches() 

2023-02-15 12:38:08 INFO:the_graph: Cached properties being calculated: combined_edges
2023-02-15 12:38:27 INFO:the_graph: Cached properties being calculated: combined_edges_assembly_specific_genes
2023-02-15 12:38:30 INFO:the_graph: Cached properties being calculated: combined_edges_genes
2023-02-15 12:39:15 INFO:the_graph: Cached properties being calculated: lower_chars_graph
2023-02-15 12:39:16 INFO:the_graph: Cached properties being calculated: get_active_ranges_of_id
2023-02-15 12:39:34 INFO:the_graph: Cached properties being calculated: available_external_databases
2023-02-15 12:39:35 INFO:the_graph: Cached properties being calculated: external_database_connection_form
2023-02-15 12:40:10 INFO:the_graph: Cached properties being calculated: available_genome_assemblies
2023-02-15 12:40:10 INFO:the_graph: Cached properties being calculated: available_external_databases_assembly
2023-02-15 12:40:11 INFO:the_graph: Cached properties being calculated: node_trios


In [45]:
idt.convert_identifier(l[0], final_database="HGNC Symbol", to_release=107, prioritize_to_one_filter=False, return_path=True)

{'target_id': ['FAM90A17', 'FAM90A16'],
 'last_node': [('ENSG00000285620.1', 'FAM90A16'),
  ('ENSG00000285720.1', 'FAM90A17')],
 'final_database': 'HGNC Symbol',
 'graph_id': 'FAM90A16P',
 'query_id': 'FAM90A16P',
 'no_corresponding': False,
 'no_conversion': False,
 'no_target': False,
 'the_path': {('FAM90A17',
   'ENSG00000285720.1'): (('FAM90A16P',
    'ENSG00000285720.1',
    0,
    107), ('ENSG00000285720.1', 'FAM90A17', 0, 107)),
  ('FAM90A16',
   'ENSG00000285620.1'): (('FAM90A16P',
    'ENSG00000285620.1',
    0,
    107), ('ENSG00000285620.1', 'FAM90A16', 0, 107))}}

In [49]:
l = ["SNORA71-2", "SNORA71", "ENSG00000201811", "SNORA60"]

In [56]:
idt.track.graph[l[2]]

AdjacencyView({'ENSG00000201811.1': {0: {'connection': {'base_ensembl_gene': {37: {79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107}, 38: {79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107}}}, 'available_releases': {79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107}}}})

In [58]:
idt.convert_identifier(l[1], final_database="HGNC Symbol", to_release=107, prioritize_to_one_filter=True, return_path=True)

{'target_id': ['SNORA71D', 'SNORA60', 'SNORA71A', 'SNORA71C'],
 'last_node': [('ENSG00000201512.1', 'SNORA71C'),
  ('ENSG00000199266.1', 'SNORA60'),
  ('ENSG00000200354.1', 'SNORA71D'),
  ('ENSG00000225091.3', 'SNORA71A')],
 'final_database': 'HGNC Symbol',
 'graph_id': 'SNORA71',
 'query_id': 'SNORA71',
 'no_corresponding': False,
 'no_conversion': False,
 'no_target': False,
 'the_path': {('SNORA71C',
   'ENSG00000201512.1'): (('SNORA71',
    'ENSG00000201512.1',
    0,
    107), ('ENSG00000201512.1', 'SNORA71C', 0, 107)),
  ('SNORA71D', 'ENSG00000200354.1'): (('SNORA71', 'ENSG00000200354.1', 0, 107),
   ('ENSG00000200354.1', 'SNORA71D', 0, 107)),
  ('SNORA60', 'ENSG00000199266.1'): (('SNORA71', 'ENSG00000199266.1', 0, 107),
   ('ENSG00000199266.1', 'SNORA60', 0, 107)),
  ('SNORA71A', 'ENSG00000225091.3'): (('SNORA71', 'ENSG00000225091.3', 0, 107),
   ('ENSG00000225091.3', 'SNORA71A', 0, 107))}}